In [1]:
from rdflib import RDF, RDFS, OWL, Namespace, Graph, Literal
import brickschema
from brickschema.namespaces import A, BRICK, UNIT
import psycopg2
from psycopg2.extras import DictCursor
from rdflib import Graph, BNode, Literal, Namespace, RDF

In [2]:
g = brickschema.Graph()
with open("abacws-building-v5.ttl", "w") as f:
    # the Turtle format strikes a balance beteween being compact and easy to read
    f.write(g.serialize(format="ttl"))

print(f"Before: {len(g)} triples")

Before: 0 triples



"""

A Brick model is a digital representation of the resources and relationships
inside a building described using the Brick schema.

A Brick model is maintained/query through the abstraction of a Graph. A Graph
consists of triples (subject, predicate, object). Subjects and objects can be
"classes" or "instances of classes" (sometimes called individuals, instances or
entities). Predicates are the "directed edges" in the graph and are sometimes
referred to as such

    +---------+        Predicate         +--------+
    | Subject |------------------------->| Object |
    +---------+                          +--------+

"""

In [2]:
g = brickschema.Graph()


Now that we have the graph object we have to decide what to put into it. At the
very least, we probably want to put in triples describing the "things" in our
building. In Brick, "things" have a name and a class, which tells us what kind
of thing they are.

The Brick schema defines a set of classes that are ready for you to use. Brick
classes includes HVAC equipment, lighting equipment, electrical equipment,
rooms and other spatial elements, zones, and points like sensors and setpoints.
You can find a list of the classes supported by Brick online:
https://brickschema.org/ontology

Before we start adding things to our Brick model, we need to define a namespace.
A "namespace" is a way of grouping information so that the names we choose for
things don't conflict with names in other Brick models (more on that later).

Namespaces are URLs; they do not have to actually point to a real web resource,
but it is of course helpful if they point to some documentation (try going to
https://brickschema.org/schema/Brick#Air_Handler_Unit as an example).

We will choose an arbitrary URL for our namespace and refer to it by the
nickname "bldg" for convenience. "bldg" is also called a "prefix".

In [ ]:
bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
bacnet = Namespace("http://data.ashrae.org/bacnet/2020#")
brick = Namespace("https://brickschema.org/schema/Brick#")
bsh = Namespace("https://brickschema.org/schema/BrickShape#")
dcterms = Namespace("http://purl.org/dc/terms/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
qudt = Namespace("http://qudt.org/schema/qudt/")
qudtqk = Namespace("http://qudt.org/vocab/quantitykind/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
ref = Namespace("https://brickschema.org/schema/Brick/ref#")
s223 = Namespace("http://data.ashrae.org/standard223#")
sdo = Namespace("http://schema.org/")
sh = Namespace("http://www.w3.org/ns/shacl#")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
sosa = Namespace("http://www.w3.org/ns/sosa/")
tag = Namespace("https://brickschema.org/schema/BrickTag#")
unit = Namespace("http://qudt.org/vocab/unit/")
vcard = Namespace("http://www.w3.org/2006/vcard/ns#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
rec = Namespace("https://w3id.org/rec#")
brickpatches = Namespace("https://w3id.org/rec/brickpatches#")
dash = Namespace("http://datashapes.org/dash#")


g.bind("bldg", bldg)
g.bind("bacnet", bacnet)
g.bind("brick", brick)
g.bind("bsh", bsh)
g.bind("dcterms", dcterms)
g.bind("owl", owl)
g.bind("qudt", qudt)
g.bind("qudtqk", qudtqk)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("ref", ref)
g.bind("s223", s223)
g.bind("sdo", sdo)
g.bind("sh", sh)
g.bind("skos", skos)
g.bind("sosa", sosa)
g.bind("tag", tag)
g.bind("unit", unit)
g.bind("vcard", vcard)
g.bind("xsd", xsd)
g.bind("rec", rec)
g.bind("brickpatches", brickpatches)
g.bind("dash", dash)

Here, we tell our graph what the Brick namespace is. This does *not* mean that
the contents of the Brick schema (which is a graph itself) get pulled in. It
simply allows us to refer to classes and relationships that are defined in the
Brick schema.

In [4]:
# brick = Namespace("https://brickschema.org/schema/Brick#")
# g.bind("brick", brick)
# RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
# g.bind("rdf", RDF)

Now we can add instances to our Brick model (building graph). We use the RDF
namespace (imported above) in addition to the BRICK and BLDG namespaces.
"RDF.type" is the "type" predicate defined within the RDF graph. The RDF.type
predicate is how we create "instances" of Brick classes

In [5]:
g.add((bldg["Abacws"], A, rec["Building"]))
g.add((bldg["Abacws"], A, rec["School"]))
# g.add((bldg[floor], A, rec[floor].))
# g.add((bldg[sensor], bldg["hasDeviceKey"], Literal("co_gas")))

In [ ]:
g.add(
    (
        bldg["Abacws"],
        rec["address"],
        Literal("Senghennydd Rd, Cardiff CF24 4AG, Wales, United Kingdom, UK"),
    )
)
g.add((bldg["Abacws"], rec["PostalAddress"], Literal("CF24 4AG")))
g.add((bldg["Abacws"], rec["architectedBy"], Literal("Cardiff University")))
g.add((bldg["Abacws"], rec["area"], Literal("10,000 sq m")))
g.add((bldg["Abacws"], rec["capacity"], Literal("about 500 people")))
g.add((bldg["Abacws"], rec["constructedBy"], Literal("Unknows. NA")))
g.add((bldg["Abacws"], rec["operatedBy"], Literal("Cardiff University")))
g.add((bldg["Abacws"], rec["ownedBy"], Literal("Cardiff University")))

In [ ]:
rooms_and_floors = {
    "Floor0": [
        "Room0.01",
        "Room0.04",
        "Room0.05",
        "Room0.06",
        "Room0.07",
        "Room0.08",
        "Room0.10",
        "Room0.17",
        "Room0.18",
        "Room0.19",
        "Room0.20",
        "Room0.29",
        "Room0.31",
        "Room0.32",
        "Room0.33",
        "Room0.34",
    ],
    "Floor1": [
        "Room1.04",
        "Room1.06",
        "Room1.07",
        "Room1.08",
        "Room1.09",
        "Room1.25",
        "Room1.26",
        "Room1.34",
        "Room1.37",
        "Room1.38",
        "Room1.39",
        "Room1.40",
        "Room1.41",
    ],
    "Floor2": [
        "Room2.01",
        "Room2.02",
        "Room2.03",
        "Room2.04",
        "Room2.05",
        "Room2.06",
        "Room2.07",
        "Room2.08",
        "Room2.09",
        "Room2.10",
        "Room2.11",
        "Room2.12",
        "Room2.13",
        "Room2.14",
        "Room2.15",
        "Room2.17",
        "Room2.18",
        "Room2.19",
        "Room2.20",
        "Room2.21",
        "Room2.22",
        "Room2.24",
        "Room2.26",
        "Room2.34",
        "Room2.35",
        "Room2.36",
        "Room2.47",
        "Room2.48",
        "Room2.49",
        "Room2.50",
        "Room2.52",
        "Room2.53",
        "Room2.54",
        "Room2.55",
        "Room2.56",
        "Room2.57",
        "Room2.58",
        "Room2.59",
        "Room2.60",
        "Room2.61",
        "Room2.62",
        "Room2.44",
        "Room2.63",
        "Room2.64",
        "Room2.65",
        "Room2.66",
        "Room2.67",
    ],
    "Floor3": [
        "Room3.01",
        "Room3.02",
        "Room3.03",
        "Room3.04",
        "Room3.05",
        "Room3.06",
        "Room3.07",
        "Room3.08",
        "Room3.09",
        "Room3.10",
        "Room3.11",
        "Room3.12",
        "Room3.13",
        "Room3.14",
        "Room3.15",
        "Room3.16",
        "Room3.17",
        "Room3.18",
        "Room3.20",
        "Room3.22",
        "Room3.26",
        "Room3.27",
        "Room3.36",
        "Room3.37",
        "Room3.38",
        "Room3.46",
        "Room3.48",
        "Room3.50",
        "Room3.51",
        "Room3.52",
        "Room3.53",
        "Room3.54",
        "Room3.56",
        "Room3.57",
        "Room3.58",
        "Room3.59",
        "Room3.60",
        "Room3.61",
        "Room3.62",
        "Room3.63",
        "Room3.64",
        "Room3.65",
        "Room3.66",
        "Room3.67",
        "Room3.68",
        "Room3.69",
        "Room3.70",
        "Room3.71",
    ],
    "Floor4": [
        "Room4.01",
        "Room4.02",
        "Room4.03",
        "Room4.05",
        "Room4.06",
        "Room4.07",
        "Room4.08",
        "Room4.09",
        "Room4.10",
        "Room4.11",
        "Room4.12",
        "Room4.13",
        "Room4.14",
        "Room4.15",
        "Room4.16",
        "Room4.18",
        "Room4.19",
        "Room4.20",
        "Room4.21",
        "Room4.22",
        "Room4.23",
        "Room4.24",
        "Room4.34",
        "Room4.35",
        "Room4.36",
        "Room4.38",
        "Room4.44",
        "Room4.47",
        "Room4.49",
        "Room4.50",
        "Room4.51",
        "Room4.52",
        "Room4.53",
        "Room4.55",
        "Room4.56",
        "Room4.57",
        "Room4.58",
        "Room4.59",
        "Room4.60",
        "Room4.61",
        "Room4.62",
        "Room4.63",
        "Room4.64",
        "Room4.65",
        "Room4.66",
        "Room4.67",
        "Room4.68",
        "Room4.69",
        "Room4.70",
        "Room4.71",
    ],
    "Floor5": [
        "Room5.01",
        "Room5.02",
        "Room5.03",
        "Room5.04",
        "Room5.05",
        "Room5.06",
        "Room5.07",
        "Room5.08",
        "Room5.09",
        "Room5.10",
        "Room5.11",
        "Room5.12",
        "Room5.13",
        "Room5.14",
        "Room5.15",
        "Room5.16",
        "Room5.17",
        "Room5.18",
        "Room5.20",
        "Room5.21",
        "Room5.22",
        "Room5.23",
        "Room5.26",
        "Room5.34",
        "Room5.35",
        "Room5.36",
        "Room5.44",
        "Room5.45",
        "Room5.48",
        "Room5.49",
        "Room5.50",
        "Room5.51",
        "Room5.52",
        "Room5.53",
        "Room5.54",
        "Room5.56",
        "Room5.57",
        "Room5.58",
        "Room5.59",
        "Room5.60",
        "Room5.61",
        "Room5.62",
        "Room5.63",
        "Room5.64",
        "Room5.65",
        "Room5.66",
        "Room5.67",
        "Room5.68",
        "Room5.69",
        "Room5.70",
        "Room5.71",
    ],
}

In [8]:
for floor, room_list in rooms_and_floors.items():
    # Use the strings in the datastructure to refer to entities in the Brick model.
    # By putting "BLDG[floor]" into the graph, we implicitly create the entity.
    g.add((bldg[floor], A, rec["Space"]))
    for room in room_list:
        g.add((bldg[room], A, rec["Room"]))
        g.add((bldg[room], brick["isPartOf"], bldg[floor]))
        g.add((bldg[floor], brick["hasPart"], bldg[room]))

In [9]:
g.serialize("abacws-building-9july.ttl", format="ttl")

<Graph identifier=N537da62b638745d391f946c5d92d8e51 (<class 'brickschema.graph.Graph'>)>

We can also add relationships between entities in our Brick model. The
brick.feeds relationship indicates a sequence between two pieces of equipment

Let's add a few more entities so the graph is more interesting. We will
implement the Brick model for the *blue* entities in the sample graph at
brickschema.org

In [ ]:
zone_and_rooms = {
    "north-Zone": [
        "Room5.01",
        "Room5.02",
        "Room5.03",
        "Room5.04",
        "Room5.05",
        "Room5.06",
        "Room5.07",
        "Room5.09",
        "Room5.10",
        "Room5.12",
        "Room5.13",
        "Room5.14",
        "Room5.15",
        "Room5.18",
        "Room5.26",
    ],
    "west-Zone": [
        "Room5.16",
        "Room5.17",
        "Room5.18",
        "Room5.08",
        "Room5.11",
        "Room5.20",
        "Room5.21",
        "Room5.22",
        "Room5.23",
        "Room5.26",
        "Room5.34",
        "Room5.35",
        "Room5.05",
    ],
    "south-Zone": [
        "Room5.36",
        "Room5.44",
        "Room5.45",
        "Room5.48",
        "Room5.49",
        "Room5.50",
        "Room5.51",
        "Room5.52",
        "Room5.53",
        "Room5.54",
        "Room5.56",
        "Room5.57",
    ],
    "east-Zone": [
        "Room5.58",
        "Room5.59",
        "Room5.60",
        "Room5.61",
        "Room5.62",
        "Room5.63",
        "Room5.64",
        "Room5.65",
        "Room5.66",
        "Room5.67",
        "Room5.68",
        "Room5.69",
        "Room5.70",
        "Room5.71",
        "Room5.44",
    ],
    "north-west-Zone": [
        "Room5.03",
        "Room5.04",
        "Room5.06",
        "Room5.07",
        "Room5.08",
        "Room5.09",
        "Room5.10",
        "Room5.12",
        "Room5.13",
        "Room5.14",
        "Room5.15",
        "Room5.16",
        "Room5.17",
        "Room5.18",
        "Room5.20",
        "Room5.21",
        "Room5.22",
        "Room5.23",
        "Room5.26",
    ],
    "north-east-Zone": [
        "Room5.01",
        "Room5.02",
        "Room5.03",
        "Room5.04",
        "Room5.06",
        "Room5.07",
        "Room5.08",
        "Room5.09",
        "Room5.10",
        "Room5.12",
        "Room5.13",
        "Room5.60",
        "Room5.61",
        "Room5.62",
        "Room5.64",
        "Room5.66",
        "Room5.67",
        "Room5.68",
        "Room5.69",
        "Room5.70",
    ],
    "south-west-Zone": [
        "Room5.34",
        "Room5.35",
        "Room5.36",
        "Room5.20",
        "Room5.21",
        "Room5.22",
        "Room5.23",
        "Room5.50",
        "Room5.51",
        "Room5.52",
        "Room5.53",
    ],
    "south-east-Zone": [
        "Room5.50",
        "Room5.51",
        "Room5.52",
        "Room5.53",
        "Room5.54",
        "Room5.56",
        "Room5.57",
        "Room5.58",
        "Room5.59",
        "Room5.60",
        "Room5.61",
        "Room5.62",
        "Room5.63",
        "Room5.64",
        "Room5.65",
        "Room5.66",
    ],
}

In [11]:
for zone, room_list in zone_and_rooms.items():
    # Use the strings in the datastructure to refer to entities in the Brick model.
    # By putting "BLDG[floor]" into the graph, we implicitly create the entity.
    g.add((bldg[zone], A, rec["Space"]))
    g.add((bldg[zone], brick["hasPart"], bldg["room_list"]))
    for room in room_list:
        g.add((bldg[room], A, brick["Room"]))
        g.add((bldg[room], rdfs["label"], Literal(room)))
        g.add((bldg[room], rec["isPartOf"], bldg[zone]))


We can "serialize" this model to a file if we want to load it into another program.


In [12]:
g.serialize("abacws-building-9july.ttl", format="ttl")

<Graph identifier=N537da62b638745d391f946c5d92d8e51 (<class 'brickschema.graph.Graph'>)>

In [13]:
# with open("abacws-building-v2.ttl", "w") as f:
#     # the Turtle format strikes a balance beteween being compact and easy to read
#     f.write(g.serialize(format="ttl"))

Adding sensors

In [ ]:
tempsen = [
    "temp5.01",
    "temp5.02",
    "temp5.03",
    "temp5.04",
    "temp5.05",
    "temp5.06",
    "temp5.07",
    "temp5.08",
    "temp5.09",
    "temp5.10",
    "temp5.11",
    "temp5.12",
    "temp5.13",
    "temp5.14",
    "temp5.15",
    "temp5.16",
    "temp5.17",
    "temp5.18",
    "temp5.19",
    "temp5.20",
    "temp5.21",
    "temp5.22",
    "temp5.23",
    "temp5.24",
    "temp5.25",
    "temp5.26",
    "temp5.27",
    "temp5.28",
    "temp5.29",
    "temp5.30",
    "temp5.31",
    "temp5.32",
    "temp5.33",
    "temp5.34",
]
humsen = [
    "hum5.01",
    "hum5.02",
    "hum5.03",
    "hum5.04",
    "hum5.05",
    "hum5.06",
    "hum5.07",
    "hum5.08",
    "hum5.09",
    "hum5.10",
    "hum5.11",
    "hum5.12",
    "hum5.13",
    "hum5.14",
    "hum5.15",
    "hum5.16",
    "hum5.17",
    "hum5.18",
    "hum5.19",
    "hum5.20",
    "hum5.21",
    "hum5.22",
    "hum5.23",
    "hum5.24",
    "hum5.25",
    "hum5.26",
    "hum5.27",
    "hum5.28",
    "hum5.29",
    "hum5.30",
    "hum5.31",
    "hum5.32",
    "hum5.33",
    "hum5.34",
]
soundsen = [
    "sound5.01",
    "sound5.02",
    "sound5.03",
    "sound5.04",
    "sound5.05",
    "sound5.06",
    "sound5.07",
    "sound5.08",
    "sound5.09",
    "sound5.10",
    "sound5.11",
    "sound5.12",
    "sound5.13",
    "sound5.14",
    "sound5.15",
    "sound5.16",
    "sound5.17",
    "sound5.18",
    "sound5.19",
    "sound5.20",
    "sound5.21",
    "sound5.22",
    "sound5.23",
    "sound5.24",
    "sound5.25",
    "sound5.26",
    "sound5.27",
    "sound5.28",
    "sound5.29",
    "sound5.30",
    "sound5.31",
    "sound5.32",
    "sound5.33",
    "sound5.34",
]
hchosen = [
    "hcho5.01",
    "hcho5.02",
    "hcho5.03",
    "hcho5.04",
    "hcho5.05",
    "hcho5.06",
    "hcho5.07",
    "hcho5.08",
    "hcho5.09",
    "hcho5.10",
    "hcho5.11",
    "hcho5.12",
    "hcho5.13",
    "hcho5.14",
    "hcho5.15",
    "hcho5.16",
    "hcho5.17",
    "hcho5.18",
    "hcho5.19",
    "hcho5.20",
    "hcho5.21",
    "hcho5.22",
    "hcho5.23",
    "hcho5.24",
    "hcho5.25",
    "hcho5.26",
    "hcho5.27",
    "hcho5.28",
    "hcho5.29",
    "hcho5.30",
    "hcho5.31",
    "hcho5.32",
    "hcho5.33",
    "hcho5.34",
]
airqsen = [
    "airq5.01",
    "airq5.02",
    "airq5.03",
    "airq5.04",
    "airq5.05",
    "airq5.06",
    "airq5.07",
    "airq5.08",
    "airq5.09",
    "airq5.10",
    "airq5.11",
    "airq5.12",
    "airq5.13",
    "airq5.14",
    "airq5.15",
    "airq5.16",
    "airq5.17",
    "airq5.18",
    "airq5.19",
    "airq5.20",
    "airq5.21",
    "airq5.22",
    "airq5.23",
    "airq5.24",
    "airq5.25",
    "airq5.26",
    "airq5.27",
    "airq5.28",
    "airq5.29",
    "airq5.30",
    "airq5.31",
    "airq5.32",
    "airq5.33",
    "airq5.34",
]
lightsen = [
    "light5.01",
    "light5.02",
    "light5.03",
    "light5.04",
    "light5.05",
    "light5.06",
    "light5.07",
    "light5.08",
    "light5.09",
    "light5.10",
    "light5.11",
    "light5.12",
    "light5.13",
    "light5.14",
    "light5.15",
    "light5.16",
    "light5.17",
    "light5.18",
    "light5.19",
    "light5.20",
    "light5.21",
    "light5.22",
    "light5.23",
    "light5.24",
    "light5.25",
    "light5.26",
    "light5.27",
    "light5.28",
    "light5.29",
    "light5.30",
    "light5.31",
    "light5.32",
    "light5.33",
    "light5.34",
]
dustsen = [
    "dust5.01",
    "dust5.02",
    "dust5.03",
    "dust5.04",
    "dust5.05",
    "dust5.06",
    "dust5.07",
    "dust5.08",
    "dust5.09",
    "dust5.10",
    "dust5.11",
    "dust5.12",
    "dust5.13",
    "dust5.14",
    "dust5.15",
    "dust5.16",
    "dust5.17",
    "dust5.18",
    "dust5.19",
    "dust5.20",
    "dust5.21",
    "dust5.22",
    "dust5.23",
    "dust5.24",
    "dust5.25",
    "dust5.26",
    "dust5.27",
    "dust5.28",
    "dust5.29",
    "dust5.30",
    "dust5.31",
    "dust5.32",
    "dust5.33",
    "dust5.34",
]
pirsen = [
    "pir5.01",
    "pir5.02",
    "pir5.03",
    "pir5.04",
    "pir5.05",
    "pir5.06",
    "pir5.07",
    "pir5.08",
    "pir5.09",
    "pir5.10",
    "pir5.11",
    "pir5.12",
    "pir5.13",
    "pir5.14",
    "pir5.15",
    "pir5.16",
    "pir5.17",
    "pir5.18",
    "pir5.19",
    "pir5.20",
    "pir5.21",
    "pir5.22",
    "pir5.23",
    "pir5.24",
    "pir5.25",
    "pir5.26",
    "pir5.27",
    "pir5.28",
    "pir5.29",
    "pir5.30",
    "pir5.31",
    "pir5.32",
    "pir5.33",
    "pir5.34",
]
mqtwosen = [
    "mqtwo5.01",
    "mqtwo5.02",
    "mqtwo5.03",
    "mqtwo5.04",
    "mqtwo5.05",
    "mqtwo5.06",
    "mqtwo5.07",
    "mqtwo5.08",
    "mqtwo5.09",
    "mqtwo5.10",
    "mqtwo5.11",
    "mqtwo5.12",
    "mqtwo5.13",
    "mqtwo5.14",
    "mqtwo5.15",
    "mqtwo5.16",
    "mqtwo5.17",
    "mqtwo5.18",
    "mqtwo5.19",
    "mqtwo5.20",
    "mqtwo5.21",
    "mqtwo5.22",
    "mqtwo5.23",
    "mqtwo5.24",
    "mqtwo5.25",
    "mqtwo5.26",
    "mqtwo5.27",
    "mqtwo5.28",
    "mqtwo5.29",
    "mqtwo5.30",
    "mqtwo5.31",
    "mqtwo5.32",
    "mqtwo5.33",
    "mqtwo5.34",
]
mqthreesen = [
    "mqthree5.01",
    "mqthree5.02",
    "mqthree5.03",
    "mqthree5.04",
    "mqthree5.05",
    "mqthree5.06",
    "mqthree5.07",
    "mqthree5.08",
    "mqthree5.09",
    "mqthree5.10",
    "mqthree5.11",
    "mqthree5.12",
    "mqthree5.13",
    "mqthree5.14",
    "mqthree5.15",
    "mqthree5.16",
    "mqthree5.17",
    "mqthree5.18",
    "mqthree5.19",
    "mqthree5.20",
    "mqthree5.21",
    "mqthree5.22",
    "mqthree5.23",
    "mqthree5.24",
    "mqthree5.25",
    "mqthree5.26",
    "mqthree5.27",
    "mqthree5.28",
    "mqthree5.29",
    "mqthree5.30",
    "mqthree5.31",
    "mqthree5.32",
    "mqthree5.33",
    "mqthree5.34",
]
mqfivesen = [
    "mqfive5.01",
    "mqfive5.02",
    "mqfive5.03",
    "mqfive5.04",
    "mqfive5.05",
    "mqfive5.06",
    "mqfive5.07",
    "mqfive5.08",
    "mqfive5.09",
    "mqfive5.10",
    "mqfive5.11",
    "mqfive5.12",
    "mqfive5.13",
    "mqfive5.14",
    "mqfive5.15",
    "mqfive5.16",
    "mqfive5.17",
    "mqfive5.18",
    "mqfive5.19",
    "mqfive5.20",
    "mqfive5.21",
    "mqfive5.22",
    "mqfive5.23",
    "mqfive5.24",
    "mqfive5.25",
    "mqfive5.26",
    "mqfive5.27",
    "mqfive5.28",
    "mqfive5.29",
    "mqfive5.30",
    "mqfive5.31",
    "mqfive5.32",
    "mqfive5.33",
    "mqfive5.34",
]
mqninesen = [
    "mqnine5.01",
    "mqnine5.02",
    "mqnine5.03",
    "mqnine5.04",
    "mqnine5.05",
    "mqnine5.06",
    "mqnine5.07",
    "mqnine5.08",
    "mqnine5.09",
    "mqnine5.10",
    "mqnine5.11",
    "mqnine5.12",
    "mqnine5.13",
    "mqnine5.14",
    "mqnine5.15",
    "mqnine5.16",
    "mqnine5.17",
    "mqnine5.18",
    "mqnine5.19",
    "mqnine5.20",
    "mqnine5.21",
    "mqnine5.22",
    "mqnine5.23",
    "mqnine5.24",
    "mqnine5.25",
    "mqnine5.26",
    "mqnine5.27",
    "mqnine5.28",
    "mqnine5.29",
    "mqnine5.30",
    "mqnine5.31",
    "mqnine5.32",
    "mqnine5.33",
    "mqnine5.34",
]
oxysen = [
    "oxy5.01",
    "oxy5.02",
    "oxy5.03",
    "oxy5.04",
    "oxy5.05",
    "oxy5.06",
    "oxy5.07",
    "oxy5.08",
    "oxy5.09",
    "oxy5.10",
    "oxy5.11",
    "oxy5.12",
    "oxy5.13",
    "oxy5.14",
    "oxy5.15",
    "oxy5.16",
    "oxy5.17",
    "oxy5.18",
    "oxy5.19",
    "oxy5.20",
    "oxy5.21",
    "oxy5.22",
    "oxy5.23",
    "oxy5.24",
    "oxy5.25",
    "oxy5.26",
    "oxy5.27",
    "oxy5.28",
    "oxy5.29",
    "oxy5.30",
    "oxy5.31",
    "oxy5.32",
    "oxy5.33",
    "oxy5.34",
]
cotwosen = [
    "cotwo5.01",
    "cotwo5.02",
    "cotwo5.03",
    "cotwo5.04",
    "cotwo5.05",
    "cotwo5.06",
    "cotwo5.07",
    "cotwo5.08",
    "cotwo5.09",
    "cotwo5.10",
    "cotwo5.11",
    "cotwo5.12",
    "cotwo5.13",
    "cotwo5.14",
    "cotwo5.15",
    "cotwo5.16",
    "cotwo5.17",
    "cotwo5.18",
    "cotwo5.19",
    "cotwo5.20",
    "cotwo5.21",
    "cotwo5.22",
    "cotwo5.23",
    "cotwo5.24",
    "cotwo5.25",
    "cotwo5.26",
    "cotwo5.27",
    "cotwo5.28",
    "cotwo5.29",
    "cotwo5.30",
    "cotwo5.31",
    "cotwo5.32",
    "cotwo5.33",
    "cotwo5.34",
]
notwosen = [
    "notwo5.01",
    "notwo5.02",
    "notwo5.03",
    "notwo5.04",
    "notwo5.05",
    "notwo5.06",
    "notwo5.07",
    "notwo5.08",
    "notwo5.09",
    "notwo5.10",
    "notwo5.11",
    "notwo5.12",
    "notwo5.13",
    "notwo5.14",
    "notwo5.15",
    "notwo5.16",
    "notwo5.17",
    "notwo5.18",
    "notwo5.19",
    "notwo5.20",
    "notwo5.21",
    "notwo5.22",
    "notwo5.23",
    "notwo5.24",
    "notwo5.25",
    "notwo5.26",
    "notwo5.27",
    "notwo5.28",
    "notwo5.29",
    "notwo5.30",
    "notwo5.31",
    "notwo5.32",
    "notwo5.33",
    "notwo5.34",
]
c2h5chsen = [
    "c2h5ch5.01",
    "c2h5ch5.02",
    "c2h5ch5.03",
    "c2h5ch5.04",
    "c2h5ch5.05",
    "c2h5ch5.06",
    "c2h5ch5.07",
    "c2h5ch5.08",
    "c2h5ch5.09",
    "c2h5ch5.10",
    "c2h5ch5.11",
    "c2h5ch5.12",
    "c2h5ch5.13",
    "c2h5ch5.14",
    "c2h5ch5.15",
    "c2h5ch5.16",
    "c2h5ch5.17",
    "c2h5ch5.18",
    "c2h5ch5.19",
    "c2h5ch5.20",
    "c2h5ch5.21",
    "c2h5ch5.22",
    "c2h5ch5.23",
    "c2h5ch5.24",
    "c2h5ch5.25",
    "c2h5ch5.26",
    "c2h5ch5.27",
    "c2h5ch5.28",
    "c2h5ch5.29",
    "c2h5ch5.30",
    "c2h5ch5.31",
    "c2h5ch5.32",
    "c2h5ch5.33",
    "c2h5ch5.34",
]
vocsen = [
    "voc5.01",
    "voc5.02",
    "voc5.03",
    "voc5.04",
    "voc5.05",
    "voc5.06",
    "voc5.07",
    "voc5.08",
    "voc5.09",
    "voc5.10",
    "voc5.11",
    "voc5.12",
    "voc5.13",
    "voc5.14",
    "voc5.15",
    "voc5.16",
    "voc5.17",
    "voc5.18",
    "voc5.19",
    "voc5.20",
    "voc5.21",
    "voc5.22",
    "voc5.23",
    "voc5.24",
    "voc5.25",
    "voc5.26",
    "voc5.27",
    "voc5.28",
    "voc5.29",
    "voc5.30",
    "voc5.31",
    "voc5.32",
    "voc5.33",
    "voc5.34",
]
cosen = [
    "co5.01",
    "co5.02",
    "co5.03",
    "co5.04",
    "co5.05",
    "co5.06",
    "co5.07",
    "co5.08",
    "co5.09",
    "co5.10",
    "co5.11",
    "co5.12",
    "co5.13",
    "co5.14",
    "co5.15",
    "co5.16",
    "co5.17",
    "co5.18",
    "co5.19",
    "co5.20",
    "co5.21",
    "co5.22",
    "co5.23",
    "co5.24",
    "co5.25",
    "co5.26",
    "co5.27",
    "co5.28",
    "co5.29",
    "co5.30",
    "co5.31",
    "co5.32",
    "co5.33",
    "co5.34",
]

In [15]:
# for i in tempsen:
#      g.add((bldg[i], A, brick["Point"]))
#      g.add((bldg[i], A, brick["Sensor"]))

In [16]:
# g.serialize("abacws-building-9july.ttl", format="ttl")

In [17]:
def add_sensor_triples(sensor_list, sensor_type, sensor_label, bldg, g):
    for i in sensor_list:
        g.add((bldg[i], A, brick[sensor_type]))
        g.add((bldg[i], rdfs["label"], Literal(sensor_label)))


# Define sensor types and their corresponding lists and labels
sensor_types = [
    ("Temperature_Sensor", tempsen, "Room Temperature Sensor"),
    ("Humidity_Sensor", humsen, "Room Humidity Sensor"),
    ("Sensor", soundsen, "Room Sound Sensor"),
    ("Formaldehyde_Sensor", hchosen, "Room hcho/Formaldehyde Sensor"),
    ("Air_Quality_Sensor", airqsen, "Room Air Quality Sensor"),
    ("Illuminance_Sensor", lightsen, "Room Light Sensor"),
    ("PM2.5_Sensor", dustsen, "Room pm 2.5 Dust Sensor"),
    ("PIR_Sensor", pirsen, "Room Occupancy Sensor"),
    ("Gas_Sensor", mqtwosen, "Room MQ2 Gas Sensor"),
    ("Gas_Sensor", mqthreesen, "Room MQ3 Gas Sensor"),
    ("Gas_Sensor", mqfivesen, "Room MQ5 Gas Sensor"),
    ("Gas_Sensor", mqninesen, "Room MQ9 Gas Sensor"),
    ("Gas_Sensor", oxysen, "Room Oxygen gas Sensor"),
    ("CO2_Sensor", cotwosen, "Room CO2 Sensor"),
    ("NO2_Level_Sensor", notwosen, "Room NO2 Sensor"),
    ("Gas_Sensor", c2h5chsen, "Room c2h5ch Sensor"),
    ("TVOC_Level_Sensor", vocsen, "Room VOC Sensor"),
    ("CO_Sensor", cosen, "Room CO Sensor"),
]

# Loop through sensor types and add triples
for sensor_type, sensor_list, sensor_label in sensor_types:
    add_sensor_triples(sensor_list, sensor_type, sensor_label, bldg, g)

# To print the resulting RDF graph
# with open("abacws-building-9july.ttl", "w") as f:
#     # the Turtle format strikes a balance beteween being compact and easy to read
#     f.write(g.serialize(format="ttl"))

In [18]:
g.serialize("abacws-building-9july.ttl", format="ttl")

<Graph identifier=N537da62b638745d391f946c5d92d8e51 (<class 'brickschema.graph.Graph'>)>

In [ ]:
# # Define connection parameters
# conn_params = {
#     'database': 'thingsboard',
#     'user': 'thingsboard',
#     'password': 'postgres',
#     'host': 'localhost',
#     'port': 5432,
# }

# # Define units for respective keys
# units = {
#     "UV_light": "UV Index",
#     "Loudness": "dB",
#     "PM1.0Atmospheric": "µg/m³",
#     "PM2.5Atmospheric": "µg/m³",
#     "PM10Atmospheric": "µg/m³",
#     "tVOC_Concentration": "ppb",
#     "CO2eq_Concentration": "ppm",
#     "MQ2_sensor_voltage": "Volts",
#     "MQ2_Rs_ratio": "",
#     "MQ2_Rs/R0_Ratio": "",
#     "MQ3_sensor_voltage": "Volts",
#     "MQ3_Rs_ratio": "",
#     "MQ3_Rs/R0_Ratio": "",
#     "HCHO_ppm": "ppm",
#     "Air_Quality": "",  # Assuming it's a qualitative measure with no specific unit
#     "Light_value": "Lux",
#     "Visible_light": "Lux",
#     "IR_light": "Lux",
#     "MQ5_sensor_voltage": "Volts",
#     "MQ5_Rs_ratio": "",
#     "MQ5_Rs/R0_Ratio": "",
#     "MQ9_sensor_voltage": "Volts",
#     "MQ9_Rs_ratio": "",
#     "MQ9_Rs/R0_Ratio": "",
#     "O2_22.10_N2_gas": "",  # Assuming it's a gas measurement with no specific unit
#     "C2H5CH_gas": "",  # Assuming it's a gas measurement with no specific unit
#     "VOC_gas": "",  # Assuming it's a gas measurement with no specific unit
#     "CO_gas": "",  # Assuming it's a gas measurement with no specific unit
#     "Co2": "",  # Assuming it's a gas measurement with no specific unit
#     "Temperature": "°C",
#     "Humidity": "%",
#     "Luminance": "cd/m²"
# }


# def process_node(node_number):
#     # Define key to URI mappings
#     key_to_uri = {
#         "airQualityValue": f"airq5.{node_number}",
#         "c2h5ch": f"C2H5CH5.{node_number}",
#         "co_gas": f"CO5.{node_number}",
#         "co2value": f"cotwo5.{node_number}",
#         "PM10Atmospheric": f"dust5.{node_number}",
#         "hchoppmvalue": f"hcho5.{node_number}",
#         "humidity": f"hum5.{node_number}",
#         "lux": f"light5.{node_number}",
#         "rsroratio5": f"mqfive5.{node_number}",
#         "rsroratio9": f"mqnine5.{node_number}",
#         "rsroratio3": f"mqthree5.{node_number}",
#         "rsroratio2": f"mqtwo5.{node_number}",
#         "NO2": f"notwo5.{node_number}",
#         "o2per": f"oxy5.{node_number}",
#         "memsvalue": f"sound5.{node_number}",
#         "tempreture": f"temp5.{node_number}",
#         "VOC": f"VOC5.{node_number}"
#     }

#     try:
#         # Connect to the PostgreSQL database
#         conn = psycopg2.connect(**conn_params)
#         cur = conn.cursor(cursor_factory=DictCursor)  # Use DictCursor to fetch results as dictionaries

#         print(f"Successfully connected to the database for node_{node_number}.")

#         for key, sensor_uri_key in key_to_uri.items():
#             # First query to get key_id
#             cur.execute("SELECT key_id FROM ts_kv_dictionary WHERE key = %s", (key,))
#             key_id_result = cur.fetchone()
#             if key_id_result:
#                 key_id = key_id_result['key_id']  # Access 'key_id' using dictionary key
#                 print("key_id found is", key_id)
#             else:
#                 print(f"No key_id found for '{key}'")
#                 continue  # Skip to next key

#             # Second query to get device id
#             cur.execute(f"SELECT id FROM device WHERE name = 'node_{node_number}'")
#             device_id_result = cur.fetchone()
#             if device_id_result:
#                 device_id = device_id_result['id']  # Access 'id' using dictionary key
#                 print("device_id found is", device_id)
#             else:
#                 print(f"No device id found for 'node_{node_number}'")
#                 continue  # Skip to next key

#             # Example sensor URI
#             sensor_uri = bldg[sensor_uri_key]

#             # Create a blank node for each external reference
#             external_reference = BNode()

#             # Add triples to the ontology
#             g.add((sensor_uri, ref.hasExternalReference, external_reference))
#             g.add((external_reference, RDF.type, ref.TimeseriesReference))
#             g.add((external_reference, ref.hasTimeseriesId, Literal(device_id)))  # Use device_id as TimeseriesId
#             g.add((external_reference, ref.hasTimeseriesKeyId, Literal(key_id)))  # Optionally add key_id as another property

#             # Add unit information if available
#             unit_value = units.get(key, "")
#             if unit_value:
#                 g.add((sensor_uri, brick.hasUnit, Literal(unit_value, datatype=unit[unit_value])))

#             # Add storage location information
#             g.add((sensor_uri, brick.storedAt, ex.db1))

#         # Add database details as a separate resource
#         g.add(bldg.db1, RDF.type, brick.postgressDB)
#         g.add((bldg.db1, brick.database, Literal('thingsboard')))
#         g.add((bldg.db1, brick.user, Literal('thingsboard')))
#         g.add((bldg.db1, brick.password, Literal('postgres')))
#         g.add((bldg.db1, brick.host, Literal('localhost')))
#         g.add((bldg.db1, brick.port, Literal('5432')))

#     except Exception as e:
#         print(f"An error occurred for node_{node_number}: {e}")

#     finally:
#         # Close the database connection
#         if conn:
#             conn.close()

# # Loop through node numbers from 01 to 34
# for i in range(1, 35):
#     node_number = f"{i:02}"  # Format the node number with leading zeros
#     process_node(node_number)

# # Serialize and save the updated ontology
# output_file = "abacws-building-9july.ttl"
# g.serialize(destination=output_file, format="turtle")

# # output_file = "timeseriesids.ttl"
# # g.serialize(destination=output_file, format="turtle")


# print(f"UUID values added for sensors in smart-building ontology. Saved to '{output_file}'.")

In [ ]:
# import psycopg2
# from psycopg2.extras import DictCursor
# from rdflib import Graph, Literal, BNode, Namespace
# from rdflib.namespace import RDF

# Define connection parameters
conn_params = {
    "database": "thingsboard",
    "user": "thingsboard",
    "password": "postgres",
    "host": "localhost",
    "port": 5432,
}

# Define units for respective keys
units = {
    "UV_light": "UV Index",
    "Loudness": "dB",
    "PM1.0Atmospheric": "µg/m³",
    "PM2.5Atmospheric": "µg/m³",
    "PM10Atmospheric": "µg/m³",
    "tVOC_Concentration": "ppb",
    "CO2eq_Concentration": "ppm",
    "MQ2_sensor_voltage": "Volts",
    "MQ2_Rs_ratio": "",
    "MQ2_Rs/R0_Ratio": "",
    "MQ3_sensor_voltage": "Volts",
    "MQ3_Rs_ratio": "",
    "MQ3_Rs/R0_Ratio": "",
    "HCHO_ppm": "ppm",
    "Air_Quality": "",
    "Light_value": "Lux",
    "Visible_light": "Lux",
    "IR_light": "Lux",
    "MQ5_sensor_voltage": "Volts",
    "MQ5_Rs_ratio": "",
    "MQ5_Rs/R0_Ratio": "",
    "MQ9_sensor_voltage": "Volts",
    "MQ9_Rs_ratio": "",
    "MQ9_Rs/R0_Ratio": "",
    "O2_22.10_N2_gas": "",
    "C2H5CH_gas": "",
    "VOC_gas": "",
    "CO_gas": "",
    "Co2": "",
    "Temperature": "°C",
    "Humidity": "%",
    "Luminance": "cd/m²",
}


def process_node(node_number):
    # Define key to URI mappings
    key_to_uri = {
        "airQualityValue": f"airq5.{node_number}",
        "c2h5ch": f"C2H5CH5.{node_number}",
        "co_gas": f"CO5.{node_number}",
        "co2value": f"cotwo5.{node_number}",
        "PM10Atmospheric": f"dust5.{node_number}",
        "hchoppmvalue": f"hcho5.{node_number}",
        "humidity": f"hum5.{node_number}",
        "lux": f"light5.{node_number}",
        "rsroratio5": f"mqfive5.{node_number}",
        "rsroratio9": f"mqnine5.{node_number}",
        "rsroratio3": f"mqthree5.{node_number}",
        "rsroratio2": f"mqtwo5.{node_number}",
        "NO2": f"notwo5.{node_number}",
        "o2per": f"oxy5.{node_number}",
        "memsvalue": f"sound5.{node_number}",
        "tempreture": f"temp5.{node_number}",
        "VOC": f"VOC5.{node_number}",
    }

    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor(
            cursor_factory=DictCursor
        )  # Use DictCursor to fetch results as dictionaries

        print(f"Successfully connected to the database for node_{node_number}.")

        for key, sensor_uri_key in key_to_uri.items():
            # First query to get key_id
            cur.execute("SELECT key_id FROM ts_kv_dictionary WHERE key = %s", (key,))
            key_id_result = cur.fetchone()
            if key_id_result:
                key_id = key_id_result["key_id"]  # Access 'key_id' using dictionary key
                print(f"key_id found for '{key}' is {key_id}")
            else:
                print(f"No key_id found for '{key}'")
                continue  # Skip to next key

            # Second query to get device id
            cur.execute(f"SELECT id FROM device WHERE name = 'node_{node_number}'")
            device_id_result = cur.fetchone()
            if device_id_result:
                device_id = device_id_result["id"]  # Access 'id' using dictionary key
                print(f"device_id found for 'node_{node_number}' is {device_id}")
            else:
                print(f"No device id found for 'node_{node_number}'")
                continue  # Skip to next key

            # Create a blank node for each external reference
            external_reference = BNode()

            # Add triples to the ontology
            g.add((bldg[sensor_uri_key], ref.hasExternalReference, external_reference))
            g.add((external_reference, RDF.type, ref.TimeseriesReference))
            g.add(
                (external_reference, ref.hasTimeseriesId, Literal(device_id))
            )  # Use device_id as TimeseriesId
            g.add(
                (external_reference, ref.hasTimeseriesKeyId, Literal(key_id))
            )  # Optionally add key_id as another property

            # Add unit information if available
            unit_value = units.get(key, "")
            if unit_value:
                g.add((bldg[sensor_uri_key], brick.hasUnit, Literal(unit_value)))

            # Add storage location information
            g.add((bldg[sensor_uri_key], brick.storedAt, bldg.db1))

        # Add database details as a separate resource
        g.add((bldg.db1, RDF.type, brick.postgressDB))
        g.add((bldg.db1, brick.database, Literal("thingsboard")))
        g.add((bldg.db1, brick.user, Literal("thingsboard")))
        g.add((bldg.db1, brick.password, Literal("postgres")))
        g.add((bldg.db1, brick.host, Literal("localhost")))
        g.add((bldg.db1, brick.port, Literal("5432")))

    except Exception as e:
        print(f"An error occurred for node_{node_number}: {e}")

    finally:
        # Close the database connection
        if conn:
            cur.close()
            conn.close()


# Loop through node numbers from 01 to 34
for i in range(1, 35):
    node_number = f"{i:02}"  # Format the node number with leading zeros
    process_node(node_number)

# Serialize and save the updated ontology
output_file = "abacws-building-9july.ttl"
g.serialize(destination=output_file, format="turtle")

print(
    f"UUID values added for sensors in smart-building ontology. Saved to '{output_file}'."
)

Successfully connected to the database for node_01.
key_id found for 'airQualityValue' is 125
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'c2h5ch' is 129
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'co_gas' is 131
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'co2value' is 132
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'PM10Atmospheric' is 77
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'hchoppmvalue' is 122
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'humidity' is 30
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'lux' is 134
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975277c1
key_id found for 'rsroratio5' is 126
device_id found for 'node_01' is 478abf30-7db2-11ee-b0f3-69bd975

In [21]:
g.serialize("abacws-building-9july.ttl", format="ttl")

<Graph identifier=N537da62b638745d391f946c5d92d8e51 (<class 'brickschema.graph.Graph'>)>

In [ ]:
temp_and_location = {
    "temp5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "temp5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "temp5.03": ["south-west-Zone"],
    "temp5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "temp5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "temp5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "temp5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "temp5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "temp5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "temp5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "temp5.11": ["south-east-Zone"],
    "temp5.12": ["north-west-Zone", "north-Zone"],
    "temp5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "temp5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "temp5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "temp5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "temp5.17": ["north-west-Zone"],
    "temp5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "temp5.19": ["south-east-Zone"],
    "temp5.20": ["south-east-Zone", "north-east-Zone"],
    "temp5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "temp5.22": ["north-west-Zone"],
    "temp5.23": ["south-east-Zone", "north-east-Zone"],
    "temp5.24": ["south-east-Zone"],
    "temp5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "temp5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "temp5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "temp5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "temp5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "temp5.30": ["north-east-Zone"],
    "temp5.31": ["north-west-Zone", "north-Zone"],
    "temp5.32": ["north-west-Zone", "north-Zone"],
    "temp5.33": ["north-west-Zone", "north-Zone"],
    "temp5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}
hum_and_location = {
    "hum5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hum5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hum5.03": ["south-west-Zone"],
    "hum5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hum5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hum5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hum5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hum5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hum5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hum5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hum5.11": ["south-east-Zone"],
    "hum5.12": ["north-west-Zone", "north-Zone"],
    "hum5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hum5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hum5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hum5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hum5.17": ["north-west-Zone"],
    "hum5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "hum5.19": ["south-east-Zone"],
    "hum5.20": ["south-east-Zone", "north-east-Zone"],
    "hum5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "hum5.22": ["north-west-Zone"],
    "hum5.23": ["south-east-Zone", "north-east-Zone"],
    "hum5.24": ["south-east-Zone"],
    "hum5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hum5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hum5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hum5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hum5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hum5.30": ["north-east-Zone"],
    "hum5.31": ["north-west-Zone", "north-Zone"],
    "hum5.32": ["north-west-Zone", "north-Zone"],
    "hum5.33": ["north-west-Zone", "north-Zone"],
    "hum5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}

sound_and_location = {
    "sound5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "sound5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "sound5.03": ["south-west-Zone"],
    "sound5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "sound5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "sound5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "sound5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "sound5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "sound5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "sound5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "sound5.11": ["south-east-Zone"],
    "sound5.12": ["north-west-Zone", "north-Zone"],
    "sound5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "sound5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "sound5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "sound5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "sound5.17": ["north-west-Zone"],
    "sound5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "sound5.19": ["south-east-Zone"],
    "sound5.20": ["south-east-Zone", "north-east-Zone"],
    "sound5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "sound5.22": ["north-west-Zone"],
    "sound5.23": ["south-east-Zone", "north-east-Zone"],
    "sound5.24": ["south-east-Zone"],
    "sound5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "sound5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "sound5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "sound5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "sound5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "sound5.30": ["north-east-Zone"],
    "sound5.31": ["north-west-Zone", "north-Zone"],
    "sound5.32": ["north-west-Zone", "north-Zone"],
    "sound5.33": ["north-west-Zone", "north-Zone"],
    "sound5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}

hcho_and_location = {
    "hcho5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hcho5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hcho5.03": ["south-west-Zone"],
    "hcho5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hcho5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hcho5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hcho5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hcho5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hcho5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hcho5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hcho5.11": ["south-east-Zone"],
    "hcho5.12": ["north-west-Zone", "north-Zone"],
    "hcho5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hcho5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hcho5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hcho5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "hcho5.17": ["north-west-Zone"],
    "hcho5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "hcho5.19": ["south-east-Zone"],
    "hcho5.20": ["south-east-Zone", "north-east-Zone"],
    "hcho5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "hcho5.22": ["north-west-Zone"],
    "hcho5.23": ["south-east-Zone", "north-east-Zone"],
    "hcho5.24": ["south-east-Zone"],
    "hcho5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hcho5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hcho5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "hcho5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "hcho5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "hcho5.30": ["north-east-Zone"],
    "hcho5.31": ["north-west-Zone", "north-Zone"],
    "hcho5.32": ["north-west-Zone", "north-Zone"],
    "hcho5.33": ["north-west-Zone", "north-Zone"],
    "hcho5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}


airq_and_location = {
    "airq5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "airq5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "airq5.03": ["south-west-Zone"],
    "airq5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "airq5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "airq5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "airq5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "airq5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "airq5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "airq5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "airq5.11": ["south-east-Zone"],
    "airq5.12": ["north-west-Zone", "north-Zone"],
    "airq5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "airq5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "airq5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "airq5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "airq5.17": ["north-west-Zone"],
    "airq5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "airq5.19": ["south-east-Zone"],
    "airq5.20": ["south-east-Zone", "north-east-Zone"],
    "airq5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "airq5.22": ["north-west-Zone"],
    "airq5.23": ["south-east-Zone", "north-east-Zone"],
    "airq5.24": ["south-east-Zone"],
    "airq5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "airq5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "airq5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "airq5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "airq5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "airq5.30": ["north-east-Zone"],
    "airq5.31": ["north-west-Zone", "north-Zone"],
    "airq5.32": ["north-west-Zone", "north-Zone"],
    "airq5.33": ["north-west-Zone", "north-Zone"],
    "airq5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}
light_and_location = {
    "light5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "light5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "light5.03": ["south-west-Zone"],
    "light5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "light5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "light5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "light5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "light5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "light5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "light5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "light5.11": ["south-east-Zone"],
    "light5.12": ["north-west-Zone", "north-Zone"],
    "light5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "light5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "light5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "light5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "light5.17": ["north-west-Zone"],
    "light5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "light5.19": ["south-east-Zone"],
    "light5.20": ["south-east-Zone", "north-east-Zone"],
    "light5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "light5.22": ["north-west-Zone"],
    "light5.23": ["south-east-Zone", "north-east-Zone"],
    "light5.24": ["south-east-Zone"],
    "light5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "light5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "light5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "light5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "light5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "light5.30": ["north-east-Zone"],
    "light5.31": ["north-west-Zone", "north-Zone"],
    "light5.32": ["north-west-Zone", "north-Zone"],
    "light5.33": ["north-west-Zone", "north-Zone"],
    "light5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}


dust_and_location = {
    "dust5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "dust5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "dust5.03": ["south-west-Zone"],
    "dust5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "dust5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "dust5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "dust5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "dust5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "dust5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "dust5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "dust5.11": ["south-east-Zone"],
    "dust5.12": ["north-west-Zone", "north-Zone"],
    "dust5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "dust5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "dust5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "dust5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "dust5.17": ["north-west-Zone"],
    "dust5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "dust5.19": ["south-east-Zone"],
    "dust5.20": ["south-east-Zone", "north-east-Zone"],
    "dust5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "dust5.22": ["north-west-Zone"],
    "dust5.23": ["south-east-Zone", "north-east-Zone"],
    "dust5.24": ["south-east-Zone"],
    "dust5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "dust5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "dust5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "dust5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "dust5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "dust5.30": ["north-east-Zone"],
    "dust5.31": ["north-west-Zone", "north-Zone"],
    "dust5.32": ["north-west-Zone", "north-Zone"],
    "dust5.33": ["north-west-Zone", "north-Zone"],
    "dust5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}

pir_and_location = {
    "pir5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "pir5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "pir5.03": ["south-west-Zone"],
    "pir5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "pir5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "pir5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "pir5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "pir5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "pir5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "pir5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "pir5.11": ["south-east-Zone"],
    "pir5.12": ["north-west-Zone", "north-Zone"],
    "pir5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "pir5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "pir5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "pir5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "pir5.17": ["north-west-Zone"],
    "pir5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "pir5.19": ["south-east-Zone"],
    "pir5.20": ["south-east-Zone", "north-east-Zone"],
    "pir5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "pir5.22": ["north-west-Zone"],
    "pir5.23": ["south-east-Zone", "north-east-Zone"],
    "pir5.24": ["south-east-Zone"],
    "pir5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "pir5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "pir5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "pir5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "pir5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "pir5.30": ["north-east-Zone"],
    "pir5.31": ["north-west-Zone", "north-Zone"],
    "pir5.32": ["north-west-Zone", "north-Zone"],
    "pir5.33": ["north-west-Zone", "north-Zone"],
    "pir5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}
mqtwo_and_location = {
    "mqtwo5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqtwo5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqtwo5.03": ["south-west-Zone"],
    "mqtwo5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqtwo5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqtwo5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqtwo5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqtwo5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqtwo5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqtwo5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqtwo5.11": ["south-east-Zone"],
    "mqtwo5.12": ["north-west-Zone", "north-Zone"],
    "mqtwo5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqtwo5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqtwo5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqtwo5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqtwo5.17": ["north-west-Zone"],
    "mqtwo5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqtwo5.19": ["south-east-Zone"],
    "mqtwo5.20": ["south-east-Zone", "north-east-Zone"],
    "mqtwo5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqtwo5.22": ["north-west-Zone"],
    "mqtwo5.23": ["south-east-Zone", "north-east-Zone"],
    "mqtwo5.24": ["south-east-Zone"],
    "mqtwo5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqtwo5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqtwo5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqtwo5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqtwo5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqtwo5.30": ["north-east-Zone"],
    "mqtwo5.31": ["north-west-Zone", "north-Zone"],
    "mqtwo5.32": ["north-west-Zone", "north-Zone"],
    "mqtwo5.33": ["north-west-Zone", "north-Zone"],
    "mqtwo5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}


mqthree_and_location = {
    "mqthree5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqthree5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqthree5.03": ["south-west-Zone"],
    "mqthree5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqthree5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqthree5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqthree5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqthree5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqthree5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqthree5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqthree5.11": ["south-east-Zone"],
    "mqthree5.12": ["north-west-Zone", "north-Zone"],
    "mqthree5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqthree5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqthree5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqthree5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqthree5.17": ["north-west-Zone"],
    "mqthree5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqthree5.19": ["south-east-Zone"],
    "mqthree5.20": ["south-east-Zone", "north-east-Zone"],
    "mqthree5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqthree5.22": ["north-west-Zone"],
    "mqthree5.23": ["south-east-Zone", "north-east-Zone"],
    "mqthree5.24": ["south-east-Zone"],
    "mqthree5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqthree5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqthree5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqthree5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqthree5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqthree5.30": ["north-east-Zone"],
    "mqthree5.31": ["north-west-Zone", "north-Zone"],
    "mqthree5.32": ["north-west-Zone", "north-Zone"],
    "mqthree5.33": ["north-west-Zone", "north-Zone"],
    "mqthree5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}
mqfive_and_location = {
    "mqfive5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqfive5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqfive5.03": ["south-west-Zone"],
    "mqfive5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqfive5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqfive5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqfive5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqfive5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqfive5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqfive5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqfive5.11": ["south-east-Zone"],
    "mqfive5.12": ["north-west-Zone", "north-Zone"],
    "mqfive5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqfive5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqfive5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqfive5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqfive5.17": ["north-west-Zone"],
    "mqfive5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqfive5.19": ["south-east-Zone"],
    "mqfive5.20": ["south-east-Zone", "north-east-Zone"],
    "mqfive5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqfive5.22": ["north-west-Zone"],
    "mqfive5.23": ["south-east-Zone", "north-east-Zone"],
    "mqfive5.24": ["south-east-Zone"],
    "mqfive5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqfive5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqfive5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqfive5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqfive5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqfive5.30": ["north-east-Zone"],
    "mqfive5.31": ["north-west-Zone", "north-Zone"],
    "mqfive5.32": ["north-west-Zone", "north-Zone"],
    "mqfive5.33": ["north-west-Zone", "north-Zone"],
    "mqfive5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}
mqnine_and_location = {
    "mqnine5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqnine5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqnine5.03": ["south-west-Zone"],
    "mqnine5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqnine5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqnine5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqnine5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqnine5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqnine5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqnine5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqnine5.11": ["south-east-Zone"],
    "mqnine5.12": ["north-west-Zone", "north-Zone"],
    "mqnine5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqnine5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqnine5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqnine5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "mqnine5.17": ["north-west-Zone"],
    "mqnine5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqnine5.19": ["south-east-Zone"],
    "mqnine5.20": ["south-east-Zone", "north-east-Zone"],
    "mqnine5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "mqnine5.22": ["north-west-Zone"],
    "mqnine5.23": ["south-east-Zone", "north-east-Zone"],
    "mqnine5.24": ["south-east-Zone"],
    "mqnine5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqnine5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqnine5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "mqnine5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "mqnine5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "mqnine5.30": ["north-east-Zone"],
    "mqnine5.31": ["north-west-Zone", "north-Zone"],
    "mqnine5.32": ["north-west-Zone", "north-Zone"],
    "mqnine5.33": ["north-west-Zone", "north-Zone"],
    "mqnine5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}


oxy_and_location = {
    "oxy5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "oxy5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "oxy5.03": ["south-west-Zone"],
    "oxy5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "oxy5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "oxy5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "oxy5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "oxy5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "oxy5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "oxy5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "oxy5.11": ["south-east-Zone"],
    "oxy5.12": ["north-west-Zone", "north-Zone"],
    "oxy5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "oxy5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "oxy5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "oxy5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "oxy5.17": ["north-west-Zone"],
    "oxy5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "oxy5.19": ["south-east-Zone"],
    "oxy5.20": ["south-east-Zone", "north-east-Zone"],
    "oxy5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "oxy5.22": ["north-west-Zone"],
    "oxy5.23": ["south-east-Zone", "north-east-Zone"],
    "oxy5.24": ["south-east-Zone"],
    "oxy5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "oxy5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "oxy5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "oxy5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "oxy5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "oxy5.30": ["north-east-Zone"],
    "oxy5.31": ["north-west-Zone", "north-Zone"],
    "oxy5.32": ["north-west-Zone", "north-Zone"],
    "oxy5.33": ["north-west-Zone", "north-Zone"],
    "oxy5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}
cotwo_and_location = {
    "cotwo5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "cotwo5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "cotwo5.03": ["south-west-Zone"],
    "cotwo5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "cotwo5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "cotwo5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "cotwo5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "cotwo5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "cotwo5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "cotwo5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "cotwo5.11": ["south-east-Zone"],
    "cotwo5.12": ["north-west-Zone", "north-Zone"],
    "cotwo5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "cotwo5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "cotwo5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "cotwo5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "cotwo5.17": ["north-west-Zone"],
    "cotwo5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "cotwo5.19": ["south-east-Zone"],
    "cotwo5.20": ["south-east-Zone", "north-east-Zone"],
    "cotwo5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "cotwo5.22": ["north-west-Zone"],
    "cotwo5.23": ["south-east-Zone", "north-east-Zone"],
    "cotwo5.24": ["south-east-Zone"],
    "cotwo5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "cotwo5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "cotwo5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "cotwo5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "cotwo5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "cotwo5.30": ["north-east-Zone"],
    "cotwo5.31": ["north-west-Zone", "north-Zone"],
    "cotwo5.32": ["north-west-Zone", "north-Zone"],
    "cotwo5.33": ["north-west-Zone", "north-Zone"],
    "cotwo5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}


notwo_and_location = {
    "notwo5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "notwo5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "notwo5.03": ["south-west-Zone"],
    "notwo5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "notwo5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "notwo5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "notwo5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "notwo5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "notwo5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "notwo5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "notwo5.11": ["south-east-Zone"],
    "notwo5.12": ["north-west-Zone", "north-Zone"],
    "notwo5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "notwo5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "notwo5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "notwo5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "notwo5.17": ["north-west-Zone"],
    "notwo5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "notwo5.19": ["south-east-Zone"],
    "notwo5.20": ["south-east-Zone", "north-east-Zone"],
    "notwo5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "notwo5.22": ["north-west-Zone"],
    "notwo5.23": ["south-east-Zone", "north-east-Zone"],
    "notwo5.24": ["south-east-Zone"],
    "notwo5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "notwo5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "notwo5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "notwo5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "notwo5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "notwo5.30": ["north-east-Zone"],
    "notwo5.31": ["north-west-Zone", "north-Zone"],
    "notwo5.32": ["north-west-Zone", "north-Zone"],
    "notwo5.33": ["north-west-Zone", "north-Zone"],
    "notwo5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}

c2h5ch_and_location = {
    "c2h5ch5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "c2h5ch5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "c2h5ch5.03": ["south-west-Zone"],
    "c2h5ch5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "c2h5ch5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "c2h5ch5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "c2h5ch5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "c2h5ch5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "c2h5ch5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "c2h5ch5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "c2h5ch5.11": ["south-east-Zone"],
    "c2h5ch5.12": ["north-west-Zone", "north-Zone"],
    "c2h5ch5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "c2h5ch5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "c2h5ch5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "c2h5ch5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "c2h5ch5.17": ["north-west-Zone"],
    "c2h5ch5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "c2h5ch5.19": ["south-east-Zone"],
    "c2h5ch5.20": ["south-east-Zone", "north-east-Zone"],
    "c2h5ch5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "c2h5ch5.22": ["north-west-Zone"],
    "c2h5ch5.23": ["south-east-Zone", "north-east-Zone"],
    "c2h5ch5.24": ["south-east-Zone"],
    "c2h5ch5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "c2h5ch5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "c2h5ch5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "c2h5ch5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "c2h5ch5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "c2h5ch5.30": ["north-east-Zone"],
    "c2h5ch5.31": ["north-west-Zone", "north-Zone"],
    "c2h5ch5.32": ["north-west-Zone", "north-Zone"],
    "c2h5ch5.33": ["north-west-Zone", "north-Zone"],
    "c2h5ch5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}
voc_and_location = {
    "voc5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "voc5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "voc5.03": ["south-west-Zone"],
    "voc5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "voc5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "voc5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "voc5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "voc5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "voc5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "voc5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "voc5.11": ["south-east-Zone"],
    "voc5.12": ["north-west-Zone", "north-Zone"],
    "voc5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "voc5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "voc5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "voc5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "voc5.17": ["north-west-Zone"],
    "voc5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "voc5.19": ["south-east-Zone"],
    "voc5.20": ["south-east-Zone", "north-east-Zone"],
    "voc5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "voc5.22": ["north-west-Zone"],
    "voc5.23": ["south-east-Zone", "north-east-Zone"],
    "voc5.24": ["south-east-Zone"],
    "voc5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "voc5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "voc5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "voc5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "voc5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "voc5.30": ["north-east-Zone"],
    "voc5.31": ["north-west-Zone", "north-Zone"],
    "voc5.32": ["north-west-Zone", "north-Zone"],
    "voc5.33": ["north-west-Zone", "north-Zone"],
    "voc5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}


co_and_location = {
    "co5.01": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "co5.02": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "co5.03": ["south-west-Zone"],
    "co5.04": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "co5.05": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "co5.06": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "co5.07": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "co5.08": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "co5.09": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "co5.10": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "co5.11": ["south-east-Zone"],
    "co5.12": ["north-west-Zone", "north-Zone"],
    "co5.13": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "co5.14": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "co5.15": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "co5.16": ["north-west-Zone", "south-west-Zone", "west-Zone"],
    "co5.17": ["north-west-Zone"],
    "co5.18": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "co5.19": ["south-east-Zone"],
    "co5.20": ["south-east-Zone", "north-east-Zone"],
    "co5.21": ["north-west-Zone", "south-east-Zone", "north-east-Zone"],
    "co5.22": ["north-west-Zone"],
    "co5.23": ["south-east-Zone", "north-east-Zone"],
    "co5.24": ["south-east-Zone"],
    "co5.25": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "co5.26": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "co5.27": ["north-west-Zone", "south-east-Zone", "north-Zone"],
    "co5.28": ["south-east-Zone", "north-east-Zone", "east-Zone"],
    "co5.29": ["south-east-Zone", "south-west-Zone", "south-Zone"],
    "co5.30": ["north-east-Zone"],
    "co5.31": ["north-west-Zone", "north-Zone"],
    "co5.32": ["north-west-Zone", "north-Zone"],
    "co5.33": ["north-west-Zone", "north-Zone"],
    "co5.34": ["Room5.65", "east-Zone", "south-east-Zone"],
}

In [ ]:
for sensor, location in temp_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in hum_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in sound_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in hcho_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in airq_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in light_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in dust_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in pir_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in mqtwo_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in mqthree_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in mqfive_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in mqnine_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in oxy_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in cotwo_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in notwo_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in c2h5ch_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, location in voc_and_location.items():
    for location in location:
        g.add((bldg[sensor], brick["isPointOf"], bldg[location]))
        g.add((bldg[sensor], brick["hasLocation"], bldg[location]))
        g.add((bldg[location], brick["hasPoint"], bldg[sensor]))
        g.add((bldg[sensor], A, brick["Sensor"]))

for sensor, locations in co_and_location.items():
    for location in locations:
        g.add((bldg[sensor], brick.isPointOf, bldg[location]))
        g.add((bldg[sensor], brick.hasLocation, bldg[location]))
        g.add((bldg[location], brick.hasPoint, bldg[sensor]))
        g.add((bldg[sensor], RDF.type, brick.Sensor))

In [24]:
g.serialize("abacws-building-9july.ttl", format="ttl")

<Graph identifier=N537da62b638745d391f946c5d92d8e51 (<class 'brickschema.graph.Graph'>)>

In [ ]:
with open("abacws-building-v5.ttl", "w") as f:
    # the Turtle format strikes a balance beteween being compact and easy to read
    f.write(g.serialize(format="ttl"))

print(f"Before: {len(g)} triples")

In [8]:
print(f"Before: {len(g)} triples")

Before: 0 triples


In [27]:
# g.expand("owlrl")
# print(f"After: {len(g)} triples")
# g.expand(profile="shacl")  # infers Brick classes from Brick tags
# print(f"After: {len(g)} triples")

In [30]:
g.expand(profile="owlrl+vbis+shacl")

C:\Users\c21054458\AppData\Roaming\Python\Python312\site-packages\brickschema\graph.py:261: UserWarning: Could not load Reasonable reasoner. Needs 'reasonable' option during install.
  warn(
C:\Users\c21054458\AppData\Roaming\Python\Python312\site-packages\brickschema\graph.py:271: UserWarning: Could not load Allegro reasoner. Needs 'allegro' option during install.
  warn(


In [28]:
print(f"Inferred graph has {len(g)} triples")

Inferred graph has 10090 triples


In [29]:
valid, _, resultsText = g.validate()
if not valid:
    print("Graph is not valid!")
    print(resultsText)

In [30]:
g.serialize("abacws-building-9july.ttl", format="ttl")

<Graph identifier=N537da62b638745d391f946c5d92d8e51 (<class 'brickschema.graph.Graph'>)>

In [30]:
# # validating using externally-defined shapes
# external = Graph()
# external.load_file("other_shapes.ttl")
# valid, _, report = g.validate(shape_graphs=[external])
# print(f"Graph is valid? {valid}")
# if not valid:
#     print(report)

In [29]:
# perform SPARQL queries on the graph
res = g.query(
    """SELECT * WHERE {
    ?room rdf:type brick:Room .
}"""
)
for row in res:
    print(row)

(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.01'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.02'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.03'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.04'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.05'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.06'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.07'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.09'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.10'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.12'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.13'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.14'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room5.15'),)

In [ ]:
import psycopg2
from psycopg2.extras import DictCursor

# Define connection parameters
conn_params = {
    "database": "thingsboard",
    "user": "thingsboard",
    "password": "postgres",
    "host": "localhost",
    "port": 5432,
}

# Define the TimeseriesId and TimeseriesKeyId
entity_id = "478abf30-7db2-11ee-b0f3-69bd975277c1"
key = 129

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor(cursor_factory=DictCursor)
    # Define the SQL query
    sql_query = """
    SELECT *
    FROM ts_kv
    WHERE entity_id = %s AND key = %s;
    """
    # Execute the query with the given parameters
    cur.execute(sql_query, (entity_id, key))

    # Fetch all results
    results = cur.fetchall()

    # Print the results
    for row in results:
        print(row)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()

['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1701284532357, None, None, 291, None, None, '8d7a430d-7afa-46c4-ad57-eb0f284f8250', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783378189, None, None, 288, None, None, 'e1a594ab-c809-4092-bbb9-1fa62bb25bfd', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783391168, None, None, 290, None, None, 'cc8bd745-25c8-4f88-982c-e9a8212ddc0f', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783403096, None, None, 287, None, None, 'd286f486-1133-4a01-bfac-c7688843094c', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783415201, None, None, 289, None, None, 'e718cdf0-7b51-443b-ba70-90c02dbb20dd', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783428184, None, None, 291, None, None, 'ea1f499b-179b-485a-af93-f796ce24f7d7', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783439920, None, None, 289, None, None, 'e7468d63-06b8-43be-a9ad-86082518312d', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 17

In [ ]:
import psycopg2
from psycopg2.extras import DictCursor
from datetime import datetime
import time

# Define connection parameters
conn_params = {
    "database": "thingsboard",
    "user": "thingsboard",
    "password": "postgres",
    "host": "localhost",
    "port": 5432,
}

# Define the entity_id and key
entity_id = "478abf30-7db2-11ee-b0f3-69bd975277c1"
key = 129

# Define the date range in dd/mm/yyyy hh:mm:ss format
start_date_str = "01/10/2023 00:00:00"
end_date_str = "03/06/2024 23:59:59"

# Convert dates to Unix timestamps
start_date_unix = int(
    time.mktime(datetime.strptime(start_date_str, "%d/%m/%Y %H:%M:%S").timetuple())
    * 1000
)
end_date_unix = int(
    time.mktime(datetime.strptime(end_date_str, "%d/%m/%Y %H:%M:%S").timetuple()) * 1000
)
print(start_date_unix)
print(end_date_unix)
try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor(cursor_factory=DictCursor)

    # Define the SQL query with date range
    sql_query = """
    SELECT *
    FROM ts_kv
    WHERE entity_id = %s AND key = %s AND ts BETWEEN %s AND %s;
    """
    # Execute the query with the given parameters
    cur.execute(sql_query, (entity_id, key, start_date_unix, end_date_unix))

    # Fetch all results
    results = cur.fetchall()

    # Print the results
    for row in results:
        print(row)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    if cur:
        cur.close()
    if conn:
        conn.close()

1696114800000
1717455599000
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1701284532357, None, None, 291, None, None, '8d7a430d-7afa-46c4-ad57-eb0f284f8250', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783378189, None, None, 288, None, None, 'e1a594ab-c809-4092-bbb9-1fa62bb25bfd', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783391168, None, None, 290, None, None, 'cc8bd745-25c8-4f88-982c-e9a8212ddc0f', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783403096, None, None, 287, None, None, 'd286f486-1133-4a01-bfac-c7688843094c', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783415201, None, None, 289, None, None, 'e718cdf0-7b51-443b-ba70-90c02dbb20dd', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783428184, None, None, 291, None, None, 'ea1f499b-179b-485a-af93-f796ce24f7d7', None]
['478abf30-7db2-11ee-b0f3-69bd975277c1', 129, 1700783439920, None, None, 289, None, None, 'e7468d63-06b8-43be-a9ad-86082518312d', None]
['478abf30-7db2-11ee

In [30]:
# # start a blocking web server with an interface for performing
# # reasoning + querying functions
# g.serve("localhost:8080")
# # now visit in http://localhost:8080

BRIKIFY

Arguments:

SOURCE: Path/URL to the source file [required]

Options:

--input-type TEXT: Supported input types: rac, table, rdf, haystack-v4

--brick PATH: Brick.ttl

--config PATH: Custom configuration file

--output PATH: Path to the output file

--serialization-format TEXT: Supported serialization formats: turtle, xml, n3, nt, pretty-xml, trix, trig and nquads [default: turtle]

--minify / --no-minify: Remove inferable triples [default: False]

--input-format TEXT: Supported input formats: xls, csv, tsv, url, turtle, xml, n3, nt, pretty-xml, trix, trig and nquads [default: turtle]

--building-prefix TEXT: Prefix for the building namespace [default: bldg]

--building-namespace TEXT: The building namespace [default: https://example.com/bldg#]

--site-prefix TEXT: Prefix for the site namespace [default: site]

--site-namespace TEXT: The site namespace [default: https://example.com/site#]

--install-completion: Install completion for the current shell.

--show-completion: Show completion for the current shell, to copy it or customize the installation.

--help: Show this message and exit.

In [ ]:
pip install brickschema[brickify]

In [ ]:
brickify --brick "brick\abacws-building-v2.ttl"

In [1]:
import rdflib

In [4]:
from rdflib import Graph


def validate_ontology(file_path):
    try:
        # Load the ontology from the Turtle file
        g = Graph()
        g.parse(file_path, format="turtle")

        # If the graph is not empty, it is considered valid
        if len(g) > 0:
            print("The ontology is valid and well-formed.")
        else:
            print("The ontology is empty or has issues. Please check your file.")
    except Exception as e:
        print(f"There was an error validating the ontology: {e}")


# Set the path to your ontology Turtle file
ontology_file_path = "./trial/dataset/abacws-building-v5.ttl"

# Call the function to validate the ontology
validate_ontology(ontology_file_path)

There was an error validating the ontology: at line 6984 of <>:
Bad syntax (Prefix ":" not bound) at ^ in:
"...b'1531ddd-8151-49db-a564-856c7009b067" ;\n        ref:storedAt '^b':database ;\n    ], [\n        a ref:TimeseriesReference ;\n   '..."


# STOP STOP STOP STOP STOP HERE - check your graph is valid

In [17]:
from rdflib import RDF, RDFS, OWL, Namespace, Graph, Literal
import brickschema
from brickschema.namespaces import A, BRICK, UNIT

ImportError: cannot import name 'brick' from 'brickschema.namespaces' (C:\Users\c21054458\AppData\Roaming\Python\Python312\site-packages\brickschema\namespaces.py)

In [8]:
g = brickschema.Graph()

In [9]:
with open("abacws-building-v5.ttl", "w") as f:
    # the Turtle format strikes a balance beteween being compact and easy to read
    f.write(g.serialize(format="ttl"))

In [10]:
valid, _, resultsText = g.validate()
if not valid:
    print("Graph is not valid!")
    print(resultsText)

Curiously enough, we haven't actually made use of the Brick schema definition
yet. The Brick schema definition contains a set of rules and definitions. These
can help with:

- ensuring that classes and relationships are being used correctly
- allowing applications and users to query the Brick schema to better
  understand a class or relationship
- providing textual definitions of classes and relationships
- inferring classes from sets of tags (like Haystack) or from behavioral
  annotations (like "sensors that measure air temperature")

To understand what the Brick schema definition can give us, lets try finding
what kinds of air temperature sensors we have in our Brick model.

In [11]:
sensors = g.query(
    """
    SELECT ?sensor WHERE {
    ?sensor rdf:type/rdfs:subClassOf* brick:Air_Temperature_Sensor
}
"""
)

This query uses the definition of the Brick class structure to find what
instances of Air_Temperature_Sensor exist as well as instances of any
*subclasses* of Air_Temperature_Sensor.  This kind of behavior is usually referred
to as "subtype polymorphism" and is analogous to what you'd find in Java.

*However*, our query returns no results because our Brick model doesn't know
what the Brick class hierarchy is yet.

In [ ]:
assert len(sensors) == 0

If we want to make use of the Brick schema definition, we need to "import" it.
You need a file called "Brick.ttl" on your computer (this can be obtained from
https://brickschema.org/resources/ or at https://github.com/BrickSchema/Brick/releases)

Assuming Brick.ttl is in the root directory of this repo, you can load it with the following.

In [ ]:
g.parse("../../Brick.ttl", format="ttl")

Now our query should execute and return one result (BLDG.VAV2-4.ZN-T)

In [ ]:
sensors = g.query(
    """SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Temperature_Sensor
}"""
)
assert len(sensors) == 1

We can also expand Brick to add additional classes. Eventually, these suggestions should
be pushed upstream and contributed back to the community (see the Contribution guidelines at
https://github.com/BrickSchema/Brick/blob/master/CONTRIBUTING.md)

For now, lets define a new kind of zone called a Fire Zone. It is good practice when defining
new classes to "attach" them to existing classes in the Brick class structure through the
RDFS.subClassOf relationship. Here's the triples we need to define a new class:

In [1]:
from rdflib import Graph

In [2]:
g = Graph()

In [3]:
g.parse("abacws-building-v5.ttl", format="turtle")

<Graph identifier=N35c7440d28924eeda3da3a80824acfb9 (<class 'rdflib.graph.Graph'>)>

In [4]:
# Count the number of triples
num_triples = len(g)

In [ ]:
print(f"Number of triples in the ontology: {num_triples}")

Number of triples in the ontology: 4899
